In [1]:
#daha önce yüklemediyseniz;
#pip install nltk
#pip install pandas
#pip install numpy
import nltk
import os
import ssl
# SSL Sertifikasını doğrulamayı devre dışı bırak
ssl._create_default_https_context = ssl._create_unverified_context
# WordNet veritabanını indir
nltk.download("wordnet")
#NLTK (Natural Language Toolkit) kütüphanesinin WordNet veritabanını indirdik.
import numpy as np
import pandas as pd
import nltk

# Gerekli veri setlerini indir
nltk.download('stopwords')     # Stopwords listesini indirir
nltk.download('punkt')         # Tokenization için gerekli veri seti
nltk.download('omw-1.4')       # Lemmatization için ek dil kaynakları (isteğe bağlı)

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from ast import literal_eval

[nltk_data] Downloading package wordnet to /Users/miray/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/miray/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/miray/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/miray/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
dosya_yolu = "/Users/miray/Downloads/Hotel_Reviews.csv"
data = pd.read_csv(dosya_yolu)
data.head()

#gerekli kütüphaneleri indirdik.
#veri setini okuttuk, ilk beş veri çıktısına baktık

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [3]:
# Hotel_Address sütunundaki son 3 kelimeyi sadece görmek
print(data['Hotel_Address'].apply(lambda x: ' '.join(x.split()[-2:])))


0         Amsterdam Netherlands
1         Amsterdam Netherlands
2         Amsterdam Netherlands
3         Amsterdam Netherlands
4         Amsterdam Netherlands
                  ...          
515733           Vienna Austria
515734           Vienna Austria
515735           Vienna Austria
515736           Vienna Austria
515737           Vienna Austria
Name: Hotel_Address, Length: 515738, dtype: object


In [4]:
# Son 2 kelimeyi ayırarak görmek ve benzersiz kombinasyonları bulmak
unique_last_2_words = data['Hotel_Address'].apply(lambda x: ' '.join(x.split()[-2:])).unique()

# Benzersiz son 2 kelime kombinasyonlarının sayısını yazdır
print(f"Farklı iki kelime kombinasyonlarının sayısı: {len(unique_last_2_words)}")

# Benzersiz ikiliyi göster
print("Benzersiz son 2 kelime kombinasyonları:")
print(unique_last_2_words)

Farklı iki kelime kombinasyonlarının sayısı: 6
Benzersiz son 2 kelime kombinasyonları:
['Amsterdam Netherlands' 'United Kingdom' 'Paris France' 'Barcelona Spain'
 'Milan Italy' 'Vienna Austria']


In [5]:
#üst kod çıktısından da anlaşacağı üzere, 
#6 farklı son 2 kelime  vardır.
#Bunlar: "United Kingdom", "Kingdom", "Netherlands", "France", "Spain", "Italy" dir.

data.Hotel_Address = data.Hotel_Address.str.replace("United Kingdom", "UK")
data.Hotel_Address = data.Hotel_Address.str.replace("Kingdom", "UK")

#Ülkeler için ayrı bir sütun oluşturmak istiyorum.
#Hotel_Addres sütununun son kelimesini alacağım şekilde ayarlamak için
#United Kingdom ve Kingdom yazılanları UK olarak değiştiyorum

data["countries"] = data.Hotel_Address.apply(lambda x: x.split(' ')[-1])
print(data.countries.unique())

#Hotel_Address sütunundaki adreslerden son kelime (ülke ismi) çıkarılıyor.
#Bu ülke isimleri yeni bir sütun olan countries içinde saklanıyor.
#Ardından, farklı ülke isimleri ekrana yazdırılıyor

['Netherlands' 'UK' 'France' 'Spain' 'Italy' 'Austria']


In [6]:
data.drop(['Additional_Number_of_Scoring', 'Review_Date', 'Reviewer_Nationality', 'Negative_Review', 'Review_Total_Negative_Word_Counts',
           'Total_Number_of_Reviews','Positive_Review','Review_Total_Positive_Word_Counts', 'Total_Number_of_Reviews_Reviewer_Has_Given',
           'Reviewer_Score', 'days_since_review', 'lat', 'lng'], axis=1, inplace=True)

# Bu kod, veri setinden belirtilen sütunları siler.
# 'data.drop()' metodu kullanılarak:
# 1. 'axis=1' parametresi sütunların silineceğini belirtir.
# 2. 'inplace=True' orijinal veri setini günceller (kopya oluşturmadan değişiklik yapar).
# Silinen sütunlar şunlardır:
# - 'Additional_Number_of_Scoring': Ek puanlama sayısı
# - 'Review_Date': İnceleme tarihi
# - 'Review_Nationality': İncelemeyi yapan kişinin milliyeti
# - 'Negative_Review': Olumsuz yorumlar
# - 'Review_Total_Negative_Word_Counts': Olumsuz yorumdaki kelime sayısı
# - 'Total_Number_of_Reviews': Toplam inceleme sayısı
# - 'Positive_Revies': (Yazım hatası olabilir, "Positive_Reviews" olması gerekiyor)
# - 'Rewiew_Total_Positive_Word_Counts': (Yazım hatası, "Review" olması gerekir) Olumlu yorumdaki kelime sayısı
# - 'Total_Number_of_Reviews_Reciewer_Has_Given': İncelemeyi yapan kişinin verdiği toplam yorum sayısı
# - 'Reviewer_Score': İncelemeyi yapan kişinin puanı
# - 'days_since_review': İncelemenin yapıldığı tarihten itibaren geçen gün sayısı
# - 'lat': Enlem bilgisi
# - 'lng': Boylam bilgisi
# Bu sütunlar silinerek veri seti küçültülür ve sadece gerekli sütunlar bırakılır.


In [7]:
# Fonksiyonun tanımı:
def impute(column):
    if isinstance(column, str):  # Eğer değer string ise
        return literal_eval(column)  # String'i listeye dönüştür
    else:
        return column  # Değer zaten listeyse olduğu gibi döner

# Tags sütunundaki her bir değere 'impute' fonksiyonu satır bazında uygulanır:
data['Tags'] = data['Tags'].apply(impute)

""" Bu dönüşüm, string formatındaki verileri Python listesi formatına çevirerek 
veri setinin tutarlı, analiz edilebilir ve makine öğrenmesi modelleri için kullanılabilir hale gelmesini sağlar.
"""
# Düzenlenmiş veri setinin ilk 5 satırını göster:
print(data.head())

                                       Hotel_Address  Average_Score  \
0   s Gravesandestraat 55 Oost 1092 AA Amsterdam ...            7.7   
1   s Gravesandestraat 55 Oost 1092 AA Amsterdam ...            7.7   
2   s Gravesandestraat 55 Oost 1092 AA Amsterdam ...            7.7   
3   s Gravesandestraat 55 Oost 1092 AA Amsterdam ...            7.7   
4   s Gravesandestraat 55 Oost 1092 AA Amsterdam ...            7.7   

    Hotel_Name                                               Tags    countries  
0  Hotel Arena  [ Leisure trip ,  Couple ,  Duplex Double Room...  Netherlands  
1  Hotel Arena  [ Leisure trip ,  Couple ,  Duplex Double Room...  Netherlands  
2  Hotel Arena  [ Leisure trip ,  Family with young children ,...  Netherlands  
3  Hotel Arena  [ Leisure trip ,  Solo traveler ,  Duplex Doub...  Netherlands  
4  Hotel Arena  [ Leisure trip ,  Couple ,  Suite ,  Stayed 2 ...  Netherlands  


In [8]:
#"Tagdata['countries'] = data['countries'].str.lower()
data['countries'] = data['countries'].str.lower()
data['Tags'] = data['Tags'].str.lower()


In [9]:
# Tags sütununda eksik (NaN) olan verileri kontrol et
print("Eksik veri sayısı:", data['Tags'].isnull().sum())

# Eksik verileri boş string ("") ile doldur
data['Tags'] = data['Tags'].fillna("")

# Tüm Tags sütununu string formatına çevir
data['Tags'] = data['Tags'].astype(str)


Eksik veri sayısı: 515738


In [14]:
"""
Şimdi konuma ve kullanıcı tarafından sağlanan açıklamaya göre
otel isimlerini öneren bir fonksiyon tanımlayalım. 
Buradaki amacımız sadece otelin ismini önermek değil, 
aynı zamanda kullanıcı derecelendirmelerine göre de sıralamak.
"""
nltk.download('punkt_tab')

# Fonksiyonun tanımı
def recommend_hotel(location, description):
    """
    Kullanıcının sağladığı konum ve açıklamaya göre otel önerir.
    Önerilen oteller benzerlik skoruna ve ortalama kullanıcı derecelendirmesine göre sıralanır.
    """
    # 1. Kullanıcı açıklamasını temizleme ve köklerine ayırma
    description = description.lower()  # Tüm harfleri küçült
    stop_words = set(stopwords.words('english'))  # İngilizce stopwords seti
    lemmatizer = WordNetLemmatizer()  # Lemmatizer objesi

    # Açıklamayı temizle, stopwords'leri kaldır ve lemmatize et
    filtered_words = [
        lemmatizer.lemmatize(word)
        for word in word_tokenize(description)
        if word.isalnum() and word not in stop_words
    ]
    filtered_set = set(filtered_words)  # Temizlenmiş kelimeleri set olarak sakla

    # 2. Belirtilen konuma göre otelleri filtrele
    location = location.lower()
    country_hotels = data[data['countries'] == location].copy()  # Yeni bir DataFrame oluştur
    country_hotels.reset_index(drop=True, inplace=True)

    # 3. Otel etiketleriyle kullanıcının açıklaması arasındaki benzerliği hesaplama
    similarities = []
    for tags in country_hotels['Tags']:
        # Etiketleri tokenize et, temizle ve lemmatize et
        tag_tokens = [
            lemmatizer.lemmatize(word)
            for word in word_tokenize(tags.lower())
            if word.isalnum() and word not in stop_words
        ]
        tag_set = set(tag_tokens)
        # Benzerlik skorunu hesapla (intersection: ortak kelime sayısı)
        similarity_score = len(filtered_set.intersection(tag_set))
        similarities.append(similarity_score)

    # 4. Benzerlik skorunu ekle ve sıralama yap
    country_hotels['similarity'] = similarities

    # Sıralama: Önce benzerliğe, sonra ortalama skora göre
    country_hotels = country_hotels.sort_values(by=['similarity', 'Average_Score'], 
                                                ascending=[False, False])

    # 5. Otel isimlerinde tekrarları kaldır ve en iyi 10 oteli öner
    country_hotels = country_hotels.drop_duplicates(subset='Hotel_Name', keep='first')
    return country_hotels[["Hotel_Name", "Average_Score", "Hotel_Address"]].head(10)



[nltk_data] Downloading package punkt_tab to /Users/miray/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [15]:
recommend_hotel('UK','I am going on a honeymoon, I need a honeymoon suite room for 3 nights')


,Hotel_Name,Average_Score,Hotel_Address
12651,Haymarket Hotel,9.6,1 Suffolk Place Westminster Borough London SW1...
144073,41,9.6,41 Buckingham Palace Road Westminster Borough ...
10275,Milestone Hotel Kensington,9.5,1 Kensington Court Kensington and Chelsea Lond...
43596,Charlotte Street Hotel,9.5,15 17 Charlotte Street Hotel Westminster Borou...
138707,The Soho Hotel,9.5,4 Richmond Mews Westminster Borough London W1D...
210217,Taj 51 Buckingham Gate Suites and Residences,9.5,Buckingham Gate Westminster Borough London SW1...
238045,Ham Yard Hotel,9.5,One Ham Yard Westminster Borough London W1D 7D...
12772,Intercontinental London The O2,9.4,1 Waterview Drive Greenwich London SE10 0TW UK
17824,Covent Garden Hotel,9.4,10 Monmouth Street Camden London WC2H 9HB UK
30937,Lansbury Heritage Hotel,9.4,117 Poplar High Street Tower Hamlets London E1...


In [16]:
recommend_hotel('italy','I am going on a honeymoon, I need a honeymoon suite room for 3 nights')


,Hotel_Name,Average_Score,Hotel_Address
6236,Excelsior Hotel Gallia Luxury Collection Hotel,9.4,Piazza Duca D Aosta 9 Central Station 20124 Mi...
1275,Palazzo Parigi Hotel Grand Spa Milano,9.3,Corso Di Porta Nuova 1 Milan City Center 20121...
10687,Room Mate Giulia,9.3,Silvio Pellico 4 Milan City Center 20121 Milan...
21777,UNA Maison Milano,9.3,Via Mazzini 4 Milan City Center 20123 Milan Italy
30199,Hotel Spadari Al Duomo,9.3,Via Spadari 11 Milan City Center 20123 Milan I...
4301,ME Milan Il Duca,9.2,Piazza della Repubblica 13 Central Station 201...
10455,The Yard Milano,9.2,Piazza XXIV Maggio 8 Milan City Center 20123 M...
12141,Mandarin Oriental Milan,9.2,Via Andegari 9 Milan City Center 20121 Milan I...
21394,Armani Hotel Milano,9.2,Via Manzoni 31 Milan City Center 20121 Milan I...
23262,Hotel Berna,9.2,Via Napo Torriani 18 Central Station 20124 Mil...


In [13]:
recommend_hotel('spain','I am going on a honeymoon, I need a honeymoon suite room for 3 nights')


,Hotel_Name,Average_Score,Hotel_Address
19517,Hotel Casa Camper,9.6,Elisabets 11 Ciutat Vella 08001 Barcelona Spain
38269,Hotel The Serras,9.6,Passeig de Colom 9 Ciutat Vella 08002 Barcelon...
41434,H10 Casa Mimosa 4 Sup,9.6,Pau Claris 179 Eixample 08037 Barcelona Spain
18094,Mercer Hotel Barcelona,9.5,Dels Lledo 7 Ciutat Vella 08003 Barcelona Spain
57,The One Barcelona GL,9.4,277 Carrer de Proven a Eixample 08037 Barcelon...
24939,Hotel Palace GL,9.4,Gran Via de les Corts Catalanes 668 Eixample 0...
29792,Catalonia Magdalenes,9.4,Magdalenes 13 15 Ciutat Vella 08002 Barcelona ...
36507,Hotel Margot House,9.4,Paseo de Gracia 46 Eixample 08007 Barcelona Spain
51267,The Wittmore Adults Only,9.4,Riudarenes 7 Ciutat Vella 08002 Barcelona Spain
53151,Catalonia Square 4 Sup,9.4,Ronda Sant Pere 9 Eixample 08010 Barcelona Spain
